# Exploratory data analysis (EDA)

Use this notebook to explore and visualize the data.

In [1]:
import os
import ibis
import ibis.selectors as s
import plotly.express as px

from python_analytics_accelerator.dag.config import (
    DATA_DIR,
    RAW_DATA_DIR,
    BRONZE,
    SILVER,
    GOLD,
)
from python_analytics_accelerator.dag.resources import Catalog

ibis.options.interactive = True

px.defaults.template = "plotly_dark"

In [2]:
catalog = Catalog()

In [3]:
catalog.list_groups()

['bronze', 'gold', 'silver']

In [4]:
catalog.list_tables(GOLD)

['gold_gh_commits',
 'gold_gh_forks',
 'gold_gh_issues',
 'gold_gh_prs',
 'gold_gh_stars',
 'gold_gh_watchers',
 'gold_pypi_downloads']

In [5]:
t = catalog.table("gold_pypi_downloads").cache()
t

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ loaded_at                  ┃ project        ┃ version      ┃ date       ┃ installer    ┃ type        ┃ country_code ┃ count ┃ start_date         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ string                     │ string         │ string       │ date       │ string       │ string      │ string       │ int64 │ string             │
├────────────────────────────┼────────────────┼──────────────┼────────────┼──────────────┼─────────────┼──────────────┼───────┼────────────────────┤
│ 2024-07-28T18:43:07.123549 │ ibis-framework │ 5.1.1.dev516 │ 2023-11-18 │ bandersnatch │ sdist       │ US           │     2 │ 2023-11-17.parquet │
│ 2024-07-28T18:43:07.123549 │ ibis-framework │ 0.11.1       │ 2017-06-30 │ bandersnatch │ sdist       │ CN           │     5 │ 2017-06-30.parquet │
│ 2024-07-28T18:43:07.123549 │ ibis-framework │ 0.11.2       │ 2022-07-01 │ bandersnatch │ sdist       │ US           │     1 │ 2022-07-01.parquet │
│ 2024-07-28T18:43:07.123549 │ ibis-framework │ 0.13.0       │ 2022-07-04 │ pip          │ sdist       │ ES           │     1 │ 2022-07-01.parquet │
│ 2024-07-28T18:43:07.123549 │ ibis-framework │ 8.0.0        │ 2024-06-05 │ pip          │ bdist_wheel │ US           │  1934 │ 2024-05-31.parquet │
│ 2024-07-28T18:43:07.123549 │ ibis-framework │ 8.0.0        │ 2024-06-06 │ Browser      │ bdist_wheel │ AU           │     1 │ 2024-05-31.parquet │
│ 2024-07-28T18:43:07.123549 │ ibis-framework │ 8.0.0        │ 2024-06-06 │ pip          │ bdist_wheel │ BR           │     8 │ 2024-05-31.parquet │
│ 2024-07-28T18:43:07.123549 │ ibis-framework │ 8.0.0        │ 2024-06-06 │ pip          │ bdist_wheel │ CN           │     1 │ 2024-05-31.parquet │
│ 2024-07-28T18:43:07.123549 │ ibis-framework │ 8.0.0        │ 2024-06-06 │ pip          │ bdist_wheel │ HK           │     2 │ 2024-05-31.parquet │
│ 2024-07-28T18:43:07.123549 │ ibis-framework │ 8.0.0        │ 2024-06-06 │ poetry       │ bdist_wheel │ IE           │     5 │ 2024-05-31.parquet │
│ …                          │ …              │ …            │ …          │ …            │ …           │ …            │     … │ …                  │
└────────────────────────────┴────────────────┴──────────────┴────────────┴──────────────┴─────────────┴──────────────┴───────┴────────────────────┘

In [ ]:
t.count()

In [ ]:
t["count"].sum()

In [ ]:
t["date"].min()

In [ ]:
t = t.mutate(date=t["date"].cast("timestamp").truncate("D"))
t = t.group_by("date").agg(downloads=t["count"].sum())
t.order_by(ibis.desc("date"))

In [ ]:
t = t.select(
    timestamp="date",
    rolling_downloads=t["downloads"]
    .sum()
    .over(ibis.window(order_by="date", preceding=28, following=0)),
)
t

In [ ]:
px.line(
    t,
    x="timestamp",
    y="rolling_downloads",
)

In [ ]:
t.to_polars()

In [ ]:
t.count()

In [ ]:
t.schema()